Emulated user 1 out of 3 with FedSyn


In [ ]:
!pip install --upgrade flwr tensorflow


import flwr as fl
import tensorflow as tf
from google.colab import drive
import numpy as np
import os
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, BatchNormalization, LeakyReLU
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam

In [ ]:
CLIENT_ID = 1

drive.mount('/content/drive', force_remount=True)
DRIVE_DATA_PATH = '***' #path to datat folder
data_path = os.path.join(DRIVE_DATA_PATH, f'client_{CLIENT_ID}_data.npz')

print(f"--- This notebook is now CLIENT #{CLIENT_ID} ---")
with np.load(data_path) as data:
    x_train = data['x'].astype(np.float32)
print(f"Successfully loaded {len(x_train)} samples for this client.")

Mounted at /content/drive
--- This notebook is now CLIENT #1 ---
Successfully loaded 20000 samples for this client.


In [ ]:

LATENT_DIM = 100
IMG_SHAPE = (28, 28, 1)

def build_generator():

    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense, Reshape, Flatten, Conv2DTranspose, LeakyReLU, BatchNormalization
    model = Sequential()

    model.add(Dense(7 * 7 * 128, input_dim=LATENT_DIM))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((7, 7, 128))) # 7x7

    model.add(Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))

    # 4. Output layer (1 channel for Black/White)
    model.add(Conv2DTranspose(1, (7, 7), padding='same', activation='tanh'))

    return model

def build_discriminator():
    """
    DCGAN Discriminator: Uses Conv2D to classify images.
    """
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, LeakyReLU
    model = Sequential()

    # 1. Convolutional Layer (Input 28x28)
    model.add(Conv2D(64, (3, 3), strides=(2, 2), padding='same', input_shape=IMG_SHAPE))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4)) # Dropout is crucial for stability

    # 2. Convolutional Layer (Downsample)
    model.add(Conv2D(64, (3, 3), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))

    # 3. Flatten and Output
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))

    return model

class FedSynClient(fl.client.NumPyClient):
    def __init__(self, x_train):
        # Normalize data to [-1, 1] for GANs
        if x_train.max() > 1.0: self.x_train = (x_train - 127.5) / 127.5
        else: self.x_train = x_train * 2.0 - 1.0
        if self.x_train.ndim == 3: self.x_train = np.expand_dims(self.x_train, axis=3)

        self.generator = build_generator()
        self.discriminator = build_discriminator()

        # Standard Adam params for DCGAN
        self.generator_optimizer = tf.keras.optimizers.Adam(0.0002, 0.5)
        self.discriminator_optimizer = tf.keras.optimizers.Adam(0.0002, 0.5)
        self.loss_fn = tf.keras.losses.BinaryCrossentropy()

    def get_parameters(self, config):
        return self.generator.get_weights()

    @tf.function
    def _train_step(self, real_images):
        batch_size = tf.shape(real_images)[0]
        noise = tf.random.normal([batch_size, LATENT_DIM])

        real_labels = tf.ones([batch_size, 1]) * 0.9

        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            generated_images = self.generator(noise, training=True)
            real_output = self.discriminator(real_images, training=True)
            fake_output = self.discriminator(generated_images, training=True)

            gen_loss = self.loss_fn(tf.ones_like(fake_output), fake_output)
            real_loss = self.loss_fn(real_labels, real_output)
            fake_loss = self.loss_fn(tf.zeros_like(fake_output), fake_output)
            disc_loss = real_loss + fake_loss

        grads_gen = gen_tape.gradient(gen_loss, self.generator.trainable_variables)
        grads_disc = disc_tape.gradient(disc_loss, self.discriminator.trainable_variables)

        self.generator_optimizer.apply_gradients(zip(grads_gen, self.generator.trainable_variables))
        self.discriminator_optimizer.apply_gradients(zip(grads_disc, self.discriminator.trainable_variables))

    def fit(self, parameters, config):
        self.generator.set_weights(parameters)

        num_local_epochs = 3
        dataset = tf.data.Dataset.from_tensor_slices(self.x_train).shuffle(len(self.x_train)).batch(32)

        print(f"Client starting local training for {num_local_epochs} epochs...")
        for epoch in range(num_local_epochs):
            for image_batch in dataset:
                self._train_step(image_batch)
        print("Client local training complete.")

        num_synthetic_samples = 5000
        noise = np.random.normal(0, 1, (num_synthetic_samples, LATENT_DIM))
        synthetic_images = self.generator.predict(noise, verbose=0)
        print(f"Client generated {len(synthetic_images)} synthetic images to send.")

        return [synthetic_images], len(synthetic_images), {}

    def evaluate(self, parameters, config):
        return 0.0, 0, {}

In [ ]:

print(f"\n--- Starting Client #{CLIENT_ID} ---")
client = FedSynClient(x_train)

fl.client.start_numpy_client(
    server_address="***", #enter server ip
    client=client,
    grpc_max_message_length=536_870_912
)
print(f"\n--- Client #{CLIENT_ID} has finished. ---")


--- Starting Client #1 ---


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
	Instead, use `flwr.client.start_client()` by ensuring you first call the `.to_client()` method as shown below: 
	flwr.client.start_client(
		ser

Client starting local training for 3 epochs...
Client local training complete.


INFO :      Sent reply
INFO:flwr:Sent reply


Client generated 500 synthetic images to send.


...

Client generated 500 synthetic images to send.


INFO :      
INFO:flwr:
INFO :      Received: train message 12f29c25-4427-4a13-8490-71de7f562175
INFO:flwr:Received: train message 12f29c25-4427-4a13-8490-71de7f562175


Client starting local training for 3 epochs...
Client local training complete.


INFO :      Sent reply
INFO:flwr:Sent reply


Client generated 500 synthetic images to send.
